In [ ]:
# Install required packages
!pip install streamlit pyngrok sqlalchemy --quiet

# Step 1: Data Preparation

In [ ]:
import pandas as pd

# Upload CSV files manually or load from Drive (here we use upload for demo)
from google.colab import files

print("Upload providers_data.csv")
uploaded1 = files.upload()
print("Upload receivers_data.csv")
uploaded2 = files.upload()
print("Upload food_listings_data.csv")
uploaded3 = files.upload()
print("Upload claims_data.csv")
uploaded4 = files.upload()

# Load CSVs
providers = pd.read_csv('dataset/providers_data.csv')
receivers = pd.read_csv('dataset/receivers_data.csv')
food_listings = pd.read_csv('dataset/food_listings_data.csv')
claims = pd.read_csv('dataset/claims_data.csv')

# Cleaning helper
def clean_text_column(df, col):
    df[col] = df[col].astype(str).str.strip().str.title()
    return df

# Clean text columns
for df, cols in [(providers, ['City', 'Type']),
                 (receivers, ['City', 'Type']),
                 (food_listings, ['Location', 'Provider_Type', 'Food_Type', 'Meal_Type'])]:
    for col in cols:
        clean_text_column(df, col)

# Convert date columns to datetime
food_listings['Expiry_Date'] = pd.to_datetime(food_listings['Expiry_Date'], errors='coerce')
claims['Timestamp'] = pd.to_datetime(claims['Timestamp'], errors='coerce')

# Validate foreign keys
invalid_providers = food_listings[~food_listings['Provider_ID'].isin(providers['Provider_ID'])]
invalid_food = claims[~claims['Food_ID'].isin(food_listings['Food_ID'])]
invalid_receivers = claims[~claims['Receiver_ID'].isin(receivers['Receiver_ID'])]

print(f"Invalid Providers in Food Listings:\n{invalid_providers[['Food_ID', 'Provider_ID']]}")
print(f"Invalid Food IDs in Claims:\n{invalid_food[['Claim_ID', 'Food_ID']]}")
print(f"Invalid Receiver IDs in Claims:\n{invalid_receivers[['Claim_ID', 'Receiver_ID']]}")

# Drop invalid references
food_listings = food_listings[food_listings['Provider_ID'].isin(providers['Provider_ID'])]
claims = claims[claims['Food_ID'].isin(food_listings['Food_ID'])]
claims = claims[claims['Receiver_ID'].isin(receivers['Receiver_ID'])]

print("Data Preparation completed.")


# Step 2: Database Creation + CRUD Setup

In [ ]:
import sqlite3
from sqlalchemy import create_engine

# Create SQLite database
conn = sqlite3.connect('dataset/food_wastage.db')
cursor = conn.cursor()

# Create tables
cursor.executescript('''
DROP TABLE IF EXISTS claims;
DROP TABLE IF EXISTS food_listings;
DROP TABLE IF EXISTS receivers;
DROP TABLE IF EXISTS providers;

CREATE TABLE providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
);

CREATE TABLE receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
);

CREATE TABLE food_listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date TEXT,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES providers(Provider_ID)
);

CREATE TABLE claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp TEXT,
    FOREIGN KEY (Food_ID) REFERENCES food_listings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES receivers(Receiver_ID)
);
''')

conn.commit()

# Insert data using pandas to_sql (via SQLAlchemy)
engine = create_engine('sqlite:///food_wastage.db')

providers.to_sql('providers', engine, if_exists='append', index=False)
receivers.to_sql('receivers', engine, if_exists='append', index=False)
food_listings.to_sql('food_listings', engine, if_exists='append', index=False)
claims.to_sql('claims', engine, if_exists='append', index=False)

print("Database created and data inserted.")


# Step 3: Data Analysis — Sample Queries

In [ ]:
queries = {
    "1. Providers and Receivers count per city": '''
        SELECT p.City,
               COUNT(DISTINCT p.Provider_ID) AS Provider_Count,
               (SELECT COUNT(DISTINCT r.Receiver_ID) FROM receivers r WHERE r.City = p.City) AS Receiver_Count
        FROM providers p
        GROUP BY p.City
        ORDER BY Provider_Count DESC;
    ''',

    "2. Top food provider type by quantity": '''
        SELECT Provider_Type, SUM(Quantity) AS Total_Quantity
        FROM food_listings
        GROUP BY Provider_Type
        ORDER BY Total_Quantity DESC
        LIMIT 5;
    ''',

    "3. Contact info of food providers in a specific city (e.g. 'New York')": '''
        SELECT Name, Contact FROM providers WHERE City = 'New York';
    ''',

    "4. Receivers who claimed the most food": '''
        SELECT r.Name, COUNT(c.Claim_ID) AS Claim_Count
        FROM receivers r JOIN claims c ON r.Receiver_ID = c.Receiver_ID
        GROUP BY r.Receiver_ID
        ORDER BY Claim_Count DESC
        LIMIT 5;
    ''',

    "5. Total quantity of food available": '''
        SELECT SUM(Quantity) AS Total_Food_Available FROM food_listings;
    ''',

    "6. City with highest number of food listings": '''
        SELECT Location, COUNT(*) AS Listings_Count
        FROM food_listings
        GROUP BY Location
        ORDER BY Listings_Count DESC
        LIMIT 5;
    ''',

    "7. Most commonly available food types": '''
        SELECT Food_Type, COUNT(*) AS Count
        FROM food_listings
        GROUP BY Food_Type
        ORDER BY Count DESC;
    ''',

    "8. Number of claims made for each food item": '''
        SELECT f.Food_Name, COUNT(c.Claim_ID) AS Claims_Made
        FROM food_listings f LEFT JOIN claims c ON f.Food_ID = c.Food_ID
        GROUP BY f.Food_ID
        ORDER BY Claims_Made DESC;
    ''',

    "9. Provider with highest number of successful food claims": '''
        SELECT p.Name, COUNT(c.Claim_ID) AS Completed_Claims
        FROM providers p JOIN food_listings f ON p.Provider_ID = f.Provider_ID
        JOIN claims c ON f.Food_ID = c.Food_ID
        WHERE c.Status = 'Completed'
        GROUP BY p.Provider_ID
        ORDER BY Completed_Claims DESC
        LIMIT 5;
    ''',

    "10. Percentage of claims by status": '''
        SELECT Status,
               ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM claims), 2) AS Percentage
        FROM claims
        GROUP BY Status;
    ''',

    "11. Average quantity of food claimed per receiver": '''
        SELECT r.Name, AVG(f.Quantity) AS Avg_Quantity_Claimed
        FROM receivers r
        JOIN claims c ON r.Receiver_ID = c.Receiver_ID
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        WHERE c.Status = 'Completed'
        GROUP BY r.Receiver_ID
        ORDER BY Avg_Quantity_Claimed DESC
        LIMIT 5;
    ''',

    "12. Most claimed meal type": '''
        SELECT f.Meal_Type, COUNT(c.Claim_ID) AS Claims_Count
        FROM food_listings f JOIN claims c ON f.Food_ID = c.Food_ID
        GROUP BY f.Meal_Type
        ORDER BY Claims_Count DESC;
    ''',

    "13. Total quantity of food donated by each provider": '''
        SELECT p.Name, SUM(f.Quantity) AS Total_Quantity_Donated
        FROM providers p JOIN food_listings f ON p.Provider_ID = f.Provider_ID
        GROUP BY p.Provider_ID
        ORDER BY Total_Quantity_Donated DESC
        LIMIT 5;
    ''',

    "14. Food items close to expiry (within 3 days)": '''
        SELECT Food_Name, Expiry_Date, Quantity, Location
        FROM food_listings
        WHERE DATE(Expiry_Date) <= DATE('now', '+3 days')
        ORDER BY Expiry_Date ASC;
    ''',

    "15. Number of claims per city": '''
        SELECT l.Location, COUNT(c.Claim_ID) AS Claims_Count
        FROM food_listings l JOIN claims c ON l.Food_ID = c.Food_ID
        GROUP BY l.Location
        ORDER BY Claims_Count DESC
        LIMIT 5;
    '''
}
for name, q in queries.items():
    print(f"\n{name}:")
    display(pd.read_sql_query(q, conn))


# Step 4: Application Development — Streamlit App


In [ ]:
%%writefile app.py
import streamlit as st
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
conn = sqlite3.connect('dataset/food_wastage.db')
c = conn.cursor()

st.title("Food Listings CRUD Operations")

# CREATE new food listing
st.header("Add New Food Listing")
with st.form("add_food"):
    food_name = st.text_input("Food Name")
    quantity = st.number_input("Quantity", min_value=1)
    expiry_date = st.date_input("Expiry Date")
    provider_id = st.number_input("Provider ID", min_value=1)
    provider_type = st.text_input("Provider Type")
    location = st.text_input("Location")
    food_type = st.text_input("Food Type")
    meal_type = st.text_input("Meal Type")
    submitted = st.form_submit_button("Add Food")

    if submitted:
        c.execute('''INSERT INTO food_listings
            (Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)''',
                  (food_name, quantity, expiry_date.strftime('%Y-%m-%d'), provider_id, provider_type, location, food_type, meal_type))
        conn.commit()
        st.success("Food listing added!")

# READ current food listings
st.header("Current Food Listings")
df = pd.read_sql_query("SELECT * FROM food_listings", conn)
st.dataframe(df)

# UPDATE food quantity example
st.header("Update Food Quantity")
food_id_to_update = st.number_input("Food ID to update", min_value=1)
new_quantity = st.number_input("New Quantity", min_value=0)
if st.button("Update Quantity"):
    c.execute("UPDATE food_listings SET Quantity=? WHERE Food_ID=?", (new_quantity, food_id_to_update))
    conn.commit()
    st.success(f"Food ID {food_id_to_update} quantity updated to {new_quantity}")

# DELETE food listing example
st.header("Delete Food Listing")
food_id_to_delete = st.number_input("Food ID to delete", min_value=1, key='delete_id')
if st.button("Delete Food Listing"):
    c.execute("DELETE FROM food_listings WHERE Food_ID=?", (food_id_to_delete,))
    conn.commit()
    st.success(f"Food ID {food_id_to_delete} deleted")

st.header("Top Providers by Quantity Donated")

query = '''
SELECT p.Name, SUM(f.Quantity) AS Total_Quantity
FROM providers p JOIN food_listings f ON p.Provider_ID = f.Provider_ID
GROUP BY p.Provider_ID
ORDER BY Total_Quantity DESC
LIMIT 10;
'''

df = pd.read_sql_query(query, sqlite3.connect('food_wastage.db'))

fig, ax = plt.subplots()
ax.barh(df['Name'], df['Total_Quantity'])
ax.invert_yaxis()
ax.set_xlabel('Quantity Donated')
ax.set_title('Top 10 Food Providers by Quantity')
st.pyplot(fig)


conn.close()


In [ ]:
!ngrok authtoken 30HonFQHqqnBhKPtK8biYGqGZj7_5hqRmrt4CgKNnWNYevU63

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print(f"Open Streamlit app here:\n{public_url}")
